In [1]:
# import common libraries
import pandas as pd # data manipulation and extraction
import numpy as np # vector operations
import matplotlib.pyplot as plt # plot library
import seaborn as sns # enhanced visual plot library
#import pingouin as pg # statistics package for running tests in dataframes
import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
#pd.options.display.float_format = '{:.2f}'.format

# Load the data
data = pd.read_csv('../notebooks/data/2024-Feb-21_data.csv')

print(f'dataframe shape: {data.shape}')

FileNotFoundError: [Errno 2] No such file or directory: '../notebooks/data/2024-Feb-21_data.csv'

In [ ]:
data.responder.value_counts()

In [ ]:
list(data.columns)

In [ ]:
meds = data.loc[:,['meds_methadone_12','responder','ct']]

meds = meds.rename(columns={'meds_methadone_12':'methadone'})

# remove rows with 0 and include ct == 1
meds = meds.query('methadone > 0 & ct == 1')

meds


In [ ]:
sns.boxplot(x='responder', y='methadone', data=meds)
plt.ylim(-100, 2600)

In [ ]:
sns.boxplot(x='responder', y='methadone', data=meds)
plt.ylim(-100, 2000)

In [ ]:
meds.methadone.mean() 

In [ ]:
sns.scatterplot(data=meds, x='methadone', y=meds.index, hue='responder')

In [ ]:
meds.methadone.hist()

# normalize data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

meds['methadone'] = scaler.fit_transform(meds[['methadone']])
meds.methadone.hist()


In [ ]:
# QQ plot for meds df
import statsmodels.api as sm

# Q-Q plot
sm.qqplot(meds.methadone, line ='q')
plt.show()

In [ ]:
data.hist('meds_methadone_24')

In [ ]:
methadone_1 = data.loc[:,['meds_methadone_1','responder','ct']]

methadone_1 = methadone_1.query('meds_methadone_1 != 0')

methadone_1.dropna(inplace=True)

methadone_1.rename(columns={'meds_methadone_1':'methadone'}, inplace=True)

methadone_1

In [ ]:
methadone_1 = data.loc[:,['meds_methadone_1','responder','ct']]

methadone_1 = methadone_1.query('meds_methadone_1 != 0')

methadone_1.dropna(inplace=True)

methadone_1.rename(columns={'meds_methadone_1':'methadone'}, inplace=True)

methadone_1

In [ ]:
ax = sns.scatterplot(data=methadone_1, x='methadone', y=methadone_1.index, hue='ct')

In [ ]:
bupe_1 = data.loc[:,['meds_buprenorphine_1','meds_buprenorphine_24']]
# use query to remove rows with 0
bupe_1 = bupe_1.query('meds_buprenorphine_1 != 0 & meds_buprenorphine_24 != 0')
bupe_1.dropna(inplace=True)

bupe_1.rename(columns={'meds_buprenorphine_1':'1','meds_buprenorphine_24':'24'}, inplace=True)

bupe_1

In [ ]:
bupe_1.loc[:,['1','24']].hist(bins=10, figsize=(10,5))

In [ ]:
# create slice to observe histograms for each week of treatment
bupe_slice = data.loc[:,'meds_buprenorphine_1':'meds_buprenorphine_24']

In [ ]:
# create a hist for a slice of all columns in meds_b
ax = bupe_slice.hist(bins=10, figsize=(20,15))
# remove title or labels
for axis in ax.flatten():
    # set title to column name
    axis.set_title(f'{axis.title.get_text()}')
    # add space between plots
    # remove y axis
    axis.get_yaxis().set_visible(False)
    # remove x axis
    axis.get_xaxis().set_visible(False)
    # remove grid
    axis.grid(False)
plt.show()

# create space between plots
plt.subplots_adjust(hspace=5, wspace=1)



In [ ]:
# density plot with area under curve shaded from 200 - 300
sns.kdeplot(methadone_1['methadone'], shade=True, color='r')
plt.axvline(200, color='k', linestyle='--')
plt.axvline(300, color='k', linestyle='--')
plt.title('Methadone 1')


In [ ]:
# calculate pdf between 200 - 300 for methadone_1
#P(a < X < b) = F(b) - F(a)
a = 200
b = 320
methadone_1_pdf = methadone_1.query('methadone >= 200 & methadone <= 320').shape[0] / methadone_1.shape[0]

print(f'P({a} < X < {b}) = {methadone_1_pdf}')


In [ ]:
responder = data.groupby(['medication','responder'])['responder'].count().to_frame().rename(columns={'responder':'count'}).reset_index()

# probability
responder['probability'] = responder.groupby('medication')['count'].transform(lambda x: x/x.sum())

responder

In [ ]:
data['comp_treatment'] = np.where(data['total_visits'] == 26, 1, 0)

treatment = data.groupby(['medication','comp_treatment'])['comp_treatment'].count().to_frame().rename(columns={'comp_treatment':'count'}).reset_index()

#probability
treatment['probability'] = treatment.groupby('comp_treatment')['count'].transform(lambda x: x/x.sum())

treatment

In [ ]:
#concat
df = pd.concat([responder, treatment], axis=1)

df.insert(4, ' ', '|')

df

## Testing for Significant Difference
In this notebook, the theme will be to test the difference between patient groups and learning what may cause someone to respond to treatment.

- <u>group1</u> - n=275 patients who responded to treatment, by meeting the predetermined abstinence window<br>

- <u>group0</u> - n=471 patients who fail to respond to treatment

*Both groups completed 24 weeks of treatment total n=746 patients<br>

The following data will be analyzed<br>

- <u>Urine Drug Tests</u> - Scheduled once per week for 8 different drug classes once every 24 weeks.  Primary output measure is for positive opioid tests with secondary measure is for positive tests for other drug classes.<br>

    <u>group1</u> - n=4805 tests<br>
<br>
    <u>group0</u> - n=14935 tests<br>
<br>
- <u>Self Reported Use</u> - Patients filled out surveys once every 4 weeks.  Each survey included instances of drug use for 8 different drug classes over the previous 30 days.  Surveys were collected at week 0 (baseline), then weeks 4, 8, 12, 16, 20 and 24.<br>
<br>
<u>Medication Doses</u> - Patients were given Methadone or Buprenorphine for treatment.  The distribution of patients is listed as follows:

- <u>group1</u> - n=141 methadone and n=134 buprenorphine<br>
<br>
- <u>group2</u> - n=250 methadone and n=211 buprenophine
 

<br>
The definition of response is abstinence within a predetermined window, where the patient shows negative urine test for opioids during the final four weeks of treatment.<br>
<br>
First we will examine the statistical difference between groups and look for evidence in the data to show that the tests are robust.  The difference in patient data should serve as a risk signal.  Helping providers improve treatment, thus improving patient response. <br>
<br>


## Helper Functions
Since the dataset is very wide with over 350 columns, we created a few functions to automate the analysis.
- `plot_drug_tests()` - Will take input for `tests` and `surveys` for 8 different `drugclasses` and produce a plot showing the sum of positive tests for each drugclass for each week of treatment.  
<br>

- `drug_test_df()` - Will take same input as `plot_drug_test()` produces the dataframe for analysis without the plot.  There is option to analyze `meds` for medication doses
<br>



In [ ]:
# create function to plot number of positive tests weekly for each drug class
def plot_drug_tests(df, type, drugclass, ax=None):
    """
    Plots the sum of drug test results for a specific drug class.

    Parameters:
    df (DataFrame): The input DataFrame containing the drug test data.
    type (str): The type of drug test, either 'test' or 'survey'.
    drugclass (str): The drug class to plot the results for.
    ax (Axes, optional): The matplotlib Axes object to plot the bar chart on. If not provided, a new Axes object will be created.

    Returns:
    ax (Axes): The matplotlib Axes object containing the bar chart.

    Raises:
    AssertionError: If the type is not 'test' or 'survey'.
    AssertionError: If the drugclass is not one of 'Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'.
    """

    # assert that type is either 'test' or 'survey'
    assert type in ['test', 'survey'], 'type must be either "test" or "survey"'

    # assert that drugclass must be include Propoxyphene, Amphetamines,Cannabinoids, Benzodiazepines, Cocaine (not case sensitive)
    assert drugclass in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'], 'drugclass must be one of "Propoxyphene", "Amphetamines", "Methamphetamine", "Cannabinoids", "Benzodiazepines", "Cocaine" or "Opiate300"'

    # filter df to total_visits == 26, this filters patients that completed treatment
    df = df[df['total_visits'] == 26]

    # create dataframe with columns for type and drug class
    df = df[[col for col in df.columns if col.startswith(type+'_') and drugclass in col]]

    # remove text and leave numbers in column names
    df.columns = [col.split('_')[-1] for col in df.columns]

    # fill nan values with 0
    df = df.fillna(0)

    # replace -5.0 with 0.0 
    df = df.replace(-5.0, 0.0)

    # plot the sum of the columns

    # set condition if axis object is not passed
    if ax is None:
        ax = plt.gca()
    sns.barplot(x=df.columns, y=df.sum(), ax=ax, palette='Blues_d')
    
    # create title
    ax.set_title(f' {drugclass} tests', fontsize=15)
    
    # remove the x axis 
    ax.set_xticklabels('')

    return ax

In [ ]:

# create function to plot number of positive tests weekly for each drug class
def drug_test_df(df, type, drugclass, ax=None):
    """
    Plots the sum of drug test results for a specific drug class.

    Parameters:
    df (DataFrame): The input DataFrame containing the drug test data.
    type (str): The type of drug test, either 'test' or 'survey'.
    drugclass (str): The drug class to plot the results for.
    ax (Axes, optional): The matplotlib Axes object to plot the bar chart on. If not provided, a new Axes object will be created.

    Returns:
    ax (Axes): The matplotlib Axes object containing the bar chart.

    Raises:
    AssertionError: If the type is not 'test' or 'survey'.
    AssertionError: If the drugclass is not one of 'Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine'.
    """

    # assert that type is either 'test' or 'survey'
    assert type in ['test', 'survey','meds'], f'{type} not a valid type for reporting'

    # assert that drugclass must be include Propoxyphene, Amphetamines,Cannabinoids, Benzodiazepines, Cocaine (not case sensitive)
    assert drugclass in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine', 'cannabis','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene','cannabis','cocaine','oxycodone','methadone','amphetamine','methamphetamine','opiates','benzodiazepines','propoxyphene','buprenorphine'],f'{drugclass} not a valid drug class'

    # filter df to total_visits == 26, this filters patients that completed treatment
    df = df[df['total_visits'] == 26]

    # create dataframe with columns for type and drug class
    df = df[[col for col in df.columns if col.startswith(type+'_') and drugclass in col]]

    # remove text and leave numbers in column names
    df.columns = [col.split('_')[-1] for col in df.columns]

    # fill nan values with 0
    df = df.fillna(0)

    # replace -5.0 with 0.0 
    df = df.replace(-5.0, 0.0)

    # set condition for aggregation by type

    if type == 'meds':
        df = df.mean()
    else:
        df = df.sum()

    df = pd.DataFrame(df)

    df = df.rename(columns={0: f'{drugclass.lower()}'})

    return df

### Before getting into the analysis, show the cumulative response rate 

In [ ]:
# add appropriate columns for plotting

# calculate the cumulative sum of responders
cumulative_response = pd.Series(data['responder']).cumsum()

# calculate the cumulative sum of non-responders
cumulative_non_response = (1 - pd.Series(data['responder'])).cumsum()

# calculate the cumulative response rate
cumulative_response_rate = cumulative_response / (cumulative_response + cumulative_non_response)

In [ ]:
# plot cumulative response curve
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(cumulative_response, label='Cumulative Responders')
ax.plot(cumulative_non_response, label='Cumulative Non-Responders')
ax.set_xlabel('')
ax.set_ylabel('Number of Patients')
ax.set_title('Cumulative Response Curve')
ax.set_xticklabels('')
ax.legend()
plt.show()


In [ ]:
# responders and non-responders
print(f'Number of responders: {cumulative_response.iloc[-1]}')
print(f'Number of non-responders: {cumulative_non_response.iloc[-1]}')

### At this point, we will segment our data into 2 patient groups

In [ ]:
# filter groups by responder class and total visits, indicating patient completed treatment
group1 = data.loc[(data.responder==1)&(data.total_visits==26)]
group0 = data.loc[(data.responder==0)&(data.total_visits==26)]

## First we will analyze the urine drug tests 

### Create dataframe for group1

In [ ]:
# create dataframe with number of drug test per drug class by treatment outcome

# these are the patients that responded to treatment

# create for loop to create a df for each drug class
for drug in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']:
 # turn each drug class into a dataframe, with aggregation of drug test results
 globals()[f'{drug}'] = drug_test_df(group1, 'test', drug)

# merge all the dataframes
drug_test_r1 = pd.concat([Opiate300, Propoxyphene, Amphetamines, Methamphetamine, Cannabinoids, Benzodiazepines, Cocaine], axis=1)

# add '-r1' to the end of each column name
drug_test_r1.columns = [f'{col}_r1' for col in drug_test_r1.columns]

drug_test_r1

### create dataframe for group0

In [ ]:
# create dataframe with number of drug test per drug class by treatment outcome

# these are the patients that did not respond to treatment

# create for loop to create a df for each drug class
for drug in ['Opiate300','Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']:
 # turn each drug class into a dataframe
 globals()[f'{drug}'] = drug_test_df(group0, 'test', drug)

# merge all the dataframes
drug_test_r0 = pd.concat([Opiate300, Propoxyphene, Amphetamines, Methamphetamine, Cannabinoids, Benzodiazepines, Cocaine], axis=1)

# add '_r1' to the end of each column name
drug_test_r0.columns = [f'{col}_r0' for col in drug_test_r0.columns]

drug_test_r0


### Test for evidence of statistical significance, in difference of test results between different patient groups

In [ ]:
# create function to test each drug class for significance between difference in tests between groups
def calculate_test_pvalues(group1, group2):
    """
    Calculate test p-values for different drug classes between two groups.

    Parameters:
    group1 (DataFrame): Dataframe containing the data for group 1.
    group2 (DataFrame): Dataframe containing the data for group 2.

    Returns:
    DataFrame: A dataframe containing the test p-values, significance, confidence intervals, cohen's d, Bayes Factor (BF10), and power for each drug class.
    """
    # import required libraries
    import pingouin as pg

    # create empty lists to store values
    pval = []
    significance = []
    ci = []
    cd_list = []  
    bf10 = []
    power = []

    # perform ttest between each drug class
    for drug in ['opiate300','propoxyphene', 'amphetamines', 'methamphetamine', 'cannabinoids', 'benzodiazepines', 'cocaine']:
        t = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['p-val'][0]
        pval.append(t)
        if t < 0.05:
            significance.append('True')
        else:
            significance.append('False')
        c = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['CI95%'][0].tolist()
        c = ', '.join(map(str, c))
        ci.append(c)
        cd = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['cohen-d'][0]
        cd_list.append(cd)  
        b = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['BF10'][0]
        bf10.append(b)
        p = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['power'][0]
        power.append(p)

    test_pvs = pd.DataFrame(
        {'drugClass':['opiate300','propoxyphene', 'amphetamines', 'methamphetamine', 'cannabinoids', 'benzodiazepines', 'cocaine'],
         'pvalue':pval,
         'significance':significance,
         'CI95%':ci,
         'cohen-d':cd_list,
         'BF10:':bf10,
         'power':power})

    return test_pvs

test_pvs = calculate_test_pvalues(drug_test_r1, drug_test_r0)

test_pvs

In [ ]:
# calculate confidence intervals for meds
meds_r1 = drug_test_df(group1, 'meds', 'buprenorphine').round(2)

meds_r1


In [ ]:
# calculate confidence intervals for meds
meds_r1 = drug_test_df(group1, 'meds', 'buprenorphine').round(2)


In [ ]:
# plot confidence intervals
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x=meds_r1.index, y='buprenorphine', data=meds_r1, ax=ax, palette='Blues_d')
ax.set_title('Buprenorphine Confidence Intervals')
plt.show()



In [ ]:
# plot pvalues on distribution
fig, ax = plt.subplots(figsize=(14, 6))
sns.barplot(x='drugClass', y='pvalue', data=test_pvs, ax=ax, palette='Blues_d')
ax.set_title('P-values for Drug Test Results')
ax.set_xlabel('Drug Class')
ax.set_ylabel('P-value')
plt.show()

### There are significant differences in all drug classes between different patient groups
Now that we've identified significant differences in clinical data, we will inspect those values further.  We expect to establish thresholds for risk with patient groups to help improve treatment outcomes.

### Now we will inspect the surveys for self reported drug use, from the same patient groups.
We will apply the same statistical rigor to see if there are significant differences in surveys from different patient groups

### create a dataframe with the sum of survey results from each different 

### starting with group1

In [ ]:
# create loop to create a df for each drug class for survey data

for drug in ['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene']:
    # turn each drug class into a dataframe
    globals()[f'{drug}'] = drug_test_df(group1, 'survey', drug)
    
# merge all the dataframes
from functools import reduce

drug_survey_r1 = reduce(lambda x, y: pd.merge(x, y, right_index=True,left_index=True), [cannabis, cocaine, oxycodone, methadone, amphetamine, crystal, opiates, benzodiazepines, propoxyphene])

# add '-r1' to the end of each column name
drug_survey_r1.columns = [f'{col}_r1' for col in drug_survey_r1.columns]

drug_survey_r1

### now to group0

In [ ]:
# create loop to create a df for each drug class for survey data

for drug in ['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene']:
    # turn each drug class into a dataframe
    globals()[f'{drug}'] = drug_test_df(group0, 'survey', drug)
    
# merge all the dataframes
from functools import reduce

drug_survey_r0 = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), [cannabis, cocaine, oxycodone, methadone, amphetamine, crystal, opiates, benzodiazepines, propoxyphene])

# add '-r1' to the end of each column name
drug_survey_r0.columns = [f'{col}_r0' for col in drug_survey_r0.columns]

# rename columns
drug_survey_r0

drug_survey_r0

In [ ]:
def calculate_survey_pvalues(group1, group2):
    
    # import required library
    import pingouin as pg

    # create empty lists to store values
    tests = []
    significance = []
    ci = [] 
    cd_list = []
    bf10 = [] 
    power = []

    # perform ttest between each drug class
    for drug in ['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene']:
        t = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['p-val'][0]
        tests.append(t)
        if t < 0.05:
            significance.append('True')
        else:
            significance.append('False')
        c = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['CI95%'][0].tolist()
        c = ', '.join(map(str, c))
        ci.append(c)
        cd = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['cohen-d'][0]
        cd_list.append(cd)  
        b = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['BF10'][0]
        bf10.append(b)
        p = pg.ttest(group1[f'{drug}_r1'], group2[f'{drug}_r0'], paired=False)['power'][0]
        power.append(p)

    survey_pvs = pd.DataFrame(
        {'drugClass':['cannabis','cocaine','oxycodone','methadone','amphetamine','crystal','opiates','benzodiazepines','propoxyphene'],
         'pvalue':tests,
         'significance':significance,
         'CI95%':ci,
         'cohen-d':cd_list,
         'BF10:':bf10,
         'power':power})

    return survey_pvs

survey_pvs = calculate_survey_pvalues(drug_survey_r1, drug_survey_r0)

survey_pvs

In [ ]:
totalgroup = group1.shape[0] + group0.shape[0]

In [ ]:
# show confidence interval for differenc in mean positive tests for each drug class
ci = survey_pvs.loc[:,['CI95%']]

# separate the confidence intervals into two columns
ci = ci['CI95%'].str.split(', ', expand=True)

ci.columns = ['lower', 'upper']

ci = ci.astype(float)

ci['upp_avg_pos_test'] = ci['lower']/totalgroup
ci['low_avg_pos_test'] = ci['upper']/totalgroup

# drugclass
ci['drugClass'] = survey_pvs['drugClass']

# move  drugclass to first column
cols = ci.columns.tolist()
cols = cols[-1:] + cols[:-1]
ci = ci[cols]

ci


In [ ]:
# show difference in avg weekly positive tests
# for each responder group 
opiates1 = drug_test_df(group1, 'test', 'Opiate300')
opiates0 = drug_test_df(group0, 'test', 'Opiate300')

opiates = pd.concat([opiates1, opiates0], axis=1)

opiates.columns = ['opiates_r1', 'opiates_r0']

opiates

## Plot Results

In [ ]:
# plot results from opiates
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(x=opiates.index, y='opiates_r1', data=opiates, ax=ax, palette='Blues_d')
sns.barplot(x=opiates.index, y='opiates_r0', data=opiates, ax=ax, palette='Reds_d', alpha=0.5)
ax.set_title('Opiates Positive Tests')
plt.show()



## Look at Change in Patterns of Drug Use for All Drug Classes

### Start with Responder 1 group
- Responder - meets predefined abstinence window - final 4 weeks of treatment

In [ ]:
# lineplot for avg weekly positive tests for each drug class
# for each responder group
fig, ax = plt.subplots(2, 3, figsize=(15, 8))  
for i, drug in enumerate(['Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']):
    plot_drug_tests(group1, 'test', drug, ax=ax.flatten()[i])
plt.tight_layout()
plt.show()

### Now responder 0 group
- Non - responder - Does not meet pre-defined abstinence window

In [ ]:
# lineplot for avg weekly positive tests for each drug class
# for each responder group
fig, ax = plt.subplots(2, 3, figsize=(15, 8))  
for i, drug in enumerate(['Propoxyphene', 'Amphetamines', 'Methamphetamine', 'Cannabinoids', 'Benzodiazepines', 'Cocaine']):
    plot_drug_tests(group0, 'test', drug, ax=ax.flatten()[i])
plt.tight_layout()
plt.show()

In [ ]:
# create 2 plots adjacent
fig, ax = plt.subplots(1, 2, figsize=(20, 8))
plot_drug_tests(group1, 'test', 'Cannabinoids', ax=ax[0])
plot_drug_tests(group0, 'test', 'Cannabinoids', ax=ax[1])
fig.suptitle('Avg Weekly Positive Drug Tests', fontsize=20)
ax[0].set_title('Responder Group 1 - Positive Drug Tests', fontsize=15)
ax[1].set_title('Responder Group 0 - Positive Drug Tests', fontsize=15)
plt.show()


### Let's examine how patients do half way through threatment?

In [ ]:
# look at weeks with higher than average positive tests
# look at self reported use and medication levels too

week12_g1 = group1.agg({'test_Opiate300_12':'sum','meds_methadone_12':np.mean,'survey_opiates_12':np.mean}).to_frame().rename(columns={0:'group1'}).round()

week12_g0 = group0.agg({'test_Opiate300_12':'sum','meds_methadone_12':np.mean,'survey_opiates_12':np.mean}).to_frame().rename(columns={0:'group0'}).round()

week12 = pd.concat([week12_g1, week12_g0], axis=1)

print(f'Rsponder group1 and group 0: Statuss at week 12')
week12

### Is there a pattern between responder groups, starting early in treatment?

In [ ]:
# average positive tests for the first 12 weeks of treatment
opiates.iloc[0:12,:].mean().to_frame().rename(columns={0:'avg_pos_tests'}).round().T

In [ ]:
# averg positive tests for the final 12 weeks of treatment
opiates.iloc[12:,:].mean().to_frame().rename(columns={0:'avg_pos_tests'}).round().T

### Patient groups seem to be goin in the right direction
- Responders who do well the first half of treatment, improve by 83% decrease in positive test for final 24 weeks
- While non-responders seem to improve by 11% decrease in positive tests

Conclusion: In general, patients who remain in treatment, show decline in positive opioid test by 11% - 83% for the 2nd half of treatment


## Next we will analyze medication doses

### First we will segment the medication doses for methadone into two patient groups

In [ ]:
# create dataframe for med doses for each patient group

# responder 1 patient group
methadone_g1 = drug_test_df(group1, 'meds', 'methadone').round(2)

# responder 0 patient group
methadone_g0 = drug_test_df(group0, 'meds', 'methadone').round(2)

# combine datasets for analysis
diff = pd.concat([methadone_g1, methadone_g0], axis=1)

# rename columns    
diff.columns = ['responder', 'non_responder']

# sample of the data
diff

### Test for significance between the different patient groups

In [ ]:
import pingouin as pg

# set significance level
alpha = 0.05

ttest = pg.ttest(diff.responder, diff.non_responder, paired=False, alternative='two-sided')

display(ttest)

if ttest['p-val'][0] < alpha:
    print('Reject the null hypothesis')
else:
    print('Fail to reject the null hypothesis.') 
    print('\n')
    print('There is no significant difference in methadone doses between the two groups')

## Visualize the difference

In [ ]:
# plot the difference using line plot and fill in the gaps
fig, ax = plt.subplots(figsize=(10, 6))
diff.plot(kind='line', ax=ax)
plt.fill_between(diff.index, diff.responder, diff.non_responder, color='skyblue', alpha=0.4)
plt.title('Methadone Doses')
plt.show()
